In [1]:
import sys
paths = [
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/config",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/ds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_intrinio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/web",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_portfolio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium/api",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/estypes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esmappings",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esindexes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/utils",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_quandl",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_snp",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/public",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_barchart",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_redis",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/models",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_twitter",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_edgar",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_fred",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_oecd",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_websocket",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/symbol_mapping",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cryptocurrency",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/user",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cbonds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_brokers"
]

In [2]:
sys.path += paths
# sys.path.append("C:\\Users\\tbui\\Documents\\Stansberry-Terminal-MkI\\services\\screener_sbanalysis_api\\")

In [3]:

from pandas.io import sql as psql

In [4]:
from lib.connected_db import PgsqlExecutor

In [11]:
research_db = {
        "host": "dev-df-postgresql.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "df_user",
        "credentials": "Piq914sT",
        "database": "datafactory"
      }
snpsource = {
        "host": "snp-instance-1-us-east-1b.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "snpx_read",
        "credentials": "Nsp563Xz",
        "database": "xpressfeed"
      }

In [12]:
class Block(object):
    _sql = None
    _values = None

    def __init__(self, db, header, template, data, ret=False):
        """
        :param db:
        :param header: is '''insert into table(col1,col2)'''
        :param template: is '''({} , {})''' or '''({col1} , {col2})'''
        :param data: list of list or list of dict
        """
        self.db = db
        self.header = header
        self.template = template
        self.data = data

        self.return_values = None
        self.ret = ret

    @property
    def values(self):
        self._values = []

        if self.is_dict() is True:
            for instance in self.data:
                self._values.append(self.template.format(**instance))
        else:
            for instance in self.data:
                self._values.append(self.template.format(*instance))

        self._values = ",".join(self._values)

        self._values = '''{}'''.format(self._values) + ON_CONFLICT
        if self.ret is True:
            return self._values + " RETURNING id "
        return self._values

    @property
    def sql(self):

        return self.header + ' values ' + self.values

    @property
    def set_statement(self):
        raise NotImplementedError

    def execute(self):
        # print (self.sql)
        if self.ret is True:
            self.return_values = self.db.fetch_all_rows(self.sql).query_data
        else:
            self.db.modify_rows(self.sql)

        return self.return_values

    def is_dict(self):
        if isinstance(self.data[0], dict):
            is_dict = True
        elif isinstance(self.data[0], list):
            is_dict = False
        else:
            raise ValueError("Either dict or list only")
        return is_dict


class BlockUpdate(Block):
    def __init__(self, db, header, template, data, keys, update_cols=None, ret=False):

        Block.__init__(self, db, header, template, data, ret=ret)

        self.header = self.header + ' update_tb '
        self.keys = keys
        self.update_cols = update_cols
        self.ret = ret

        assert isinstance(keys, list), 'Must provide list of keys for update'

        self.check_keys(keys)

        assert isinstance(self.data[0], dict), 'Data must be a dictionary'

    def check_keys(self, keys):
        for col in keys:
            if col not in self.data[0]:
                raise ValueError("{} is Invalid key".format(col))

    @property
    def set_statement(self):

        if self.update_cols is None:
            self.update_cols = [col for col in self.data[0] if col not in self.keys]

        ret_set_stm = ''' SET {}  FROM '''

        modified_updates = []
        for col in self.update_cols:
            modified_updates.append('{col}=data.{col}'.format(col=col))

        modified_updates = ','.join(modified_updates)
        return ret_set_stm.format(modified_updates)

    @property
    def data_statement(self):
        ret_data_statement = ''' AS data({}) WHERE '''
        ordered_cols = []
        ordered_template = self.template.split(',')[::-1]
        while len(ordered_template) > 0:
            model = ordered_template.pop()
            for col in self.data[0].keys():
                if col in model:
                    ordered_cols.append(col)

        ordered_cols = ','.join(ordered_cols)
        return ret_data_statement.format(ordered_cols)

    @property
    def conditional_statement(self):

        conditional_cols = []
        for key in self.keys:
            conditional_cols.append('update_tb.{key}=data.{key}'.format(key=key))
        if self.ret is False:
            return ' AND '.join(conditional_cols)

        return_statement = ','.join(['update_tb.{}'.format(col) for col in self.data[0].keys()])
        return ' AND '.join(conditional_cols) + ' RETURNING ' + return_statement

    @property
    def values(self):
        ret_values = ''' (VALUES {}) '''
        values = []

        if self.is_dict() is True:
            for instance in self.data:
                values.append(self.template.format(**instance))
        else:
            for instance in self.data:
                values.append(self.template.format(*instance))

        values = ",".join(values)

        values = '''{}'''.format(values)
        return ret_values.format(values)

    @property
    def sql(self):
        return self.header + self.set_statement +\
               self.values + self.data_statement + \
               self.conditional_statement


class BlockList(Block):
    def __init__(self, db, header, template, data, ret=False):
        Block.__init__(self, db, header, template, data, ret=ret)

    @property
    def values(self):
        temp_values = [self.template] * len(self.data)
        return ",".join(temp_values)

    @property
    def args(self):
        ret = []
        for row in self.data:
            ret += row
        return ret

    def execute(self):
        self.db.execute_row(self.sql, *self.args)




class BulkDb(object):

    _divider = None

    def __init__(self, db):
        self.db = db


    def update(self, header, template, data, keys, update_cols=None, ret=False, block=3000):
        """
        :param data: data must be list or iter
        header = '''Update test'''
        template = ''' ({id}, {col1}, {col2})'''
        bulk_insert = BulkDb(db=db3)
        ret_data = bulk_insert.update(header=header, template=template,
                    data=update_list, keys=['id'],
                    update_cols= ['col2'],block=10, ret=True)
        """
        assert isinstance(data, list), "Data must be a list"

        divider = len(data) // block
        start = 0
        end = 0

        return_values = None

        if ret is True:
            return_values = []

        for index in range(0, divider + 1):
            end = start + block
            if end >= len(data):
                end = len(data)
            if start == end:
                continue

            exec_block = BlockUpdate(db=self.db, header=header, template=template, data=data[start:end],
                                     keys=keys, update_cols=update_cols, ret=ret)
            if ret is True:
                return_values += exec_block.execute()
            else:
                exec_block.execute()
            start = end

        return return_values

    def insert_list(self, header, template, data, block=3000):
        """
        :param data: data must be list of iter
        with list
        header = '''Insert into test(col1, col2)'''
        template = ''' (%s, %s)'''
        bulk_insert = BulkDb(db=db3 )
        bulk_insert.insert(header=header, template=template, data=list_data, block=10)
        """
        assert isinstance(data, list), "Data must be a list"

        divider = len(data) // block
        start = 0
        end = 0
        return_values = None
        for index in range(0, divider + 1):
            end = start + block
            if end >= len(data):
                end = len(data)
            if start == end:
                continue
            exec_block = BlockList(db=self.db, header=header, template=template, data=data[start:end])
            exec_block.execute()
            start = end
        return return_values



    def insert(self, header, template, data, block=3000, ret=False):
        """
        :param data: data must be list of iter
        with list
        header = '''Insert into test(col1, col2)'''
        template = ''' ({}, {})'''
        bulk_insert = BulkDb(db=db3 )
        bulk_insert.insert(header=header, template=template, data=list_data, block=10)
        with dict
        header = '''Insert into test(col1, col2)'''
        template = ''' ({col1}, {col2})'''
        bulk_insert = BulkDb(db=db3)
        ret_data = bulk_insert.insert(header=header, template=template, data=list_data, block=10, ret=True)
        """
        assert isinstance(data, list), "Data must be a list"

        divider = len(data) // block
        start = 0
        end = 0
        return_values = None
        if ret is True:
            return_values = []

        for index in range(0, divider + 1):

            end = start + block
            if end >= len(data):
                end = len(data)
            if start == end:
                continue
            exec_block = Block(db=self.db, header=header, template=template, data=data[start:end], ret=ret)
            if ret is True:
                return_values += exec_block.execute()
            else:
                exec_block.execute()
            start = end
        return return_values

In [13]:


db_data = {
        "host": "136.34.106.138",
        "port": 5432,
        "user": "postgres",
        "credentials": "4n89o3J824",
        "database": "stock"
      }

test_connect = PgsqlExecutor(db_data)
bulk_db = BulkDb(test_connect)

In [14]:
sql = """
select * from model.dcf_values  limit 10
"""
data = test_connect.get_dataframe(sql)
data.head()

Empty DataFrame
Columns: [id, symbol, exchangesymbol, tradingitemid, is_active, current_price, market_growth_rate, current_growth_rate, adjusted_growth_rate, variance, fiscal_year, total_years, market_dcf, dcf, adjusted_dcf, error, created]
Index: []

In [9]:
sql = """

drop table model.dcf_values;

    CREATE TABLE model.dcf_values(
        id serial NOT NULL PRIMARY KEY,
        symbol character varying(10) NOT NULL,
        exchangesymbol character varying(50),
        tradingitemid integer,
        is_active integer default 1,
        current_price numeric,
        market_growth_rate numeric default 0.095,
        current_growth_rate numeric,
        adjusted_growth_rate numeric,
        variance numeric,
        fiscal_year integer,
        total_years integer,
        market_dcf numeric,
        dcf numeric,
        adjusted_dcf numeric,
        error text,
        created timestamp without time zone DEFAULT now()

    );

    create index ind_dcf_values_symbol_idx on model.dcf_values(symbol);
	create index ind_dcf_values_symbol_fiscal_year_idx on model.dcf_values(symbol, fiscal_year);
"""
test_connect.modify_rows(sql)

ExecutionResults(query_data=None, row_count=-1)

In [ ]:
db= PgsqlExecutor(research_db)

In [ ]:
sql = """
select * from model.dcf_values where is_active = 1
"""
data = db.fetch_all_rows(sql)

data = [list(row[1:]) for row in data.query_data]

In [8]:
import requests

In [9]:
url = "https://api.nasdaq.com/api/calendar/earnings?date=2020-06-09"

In [10]:
response = requests.get(url)
reponse_values = response.json()['data']['rows']

In [11]:
for row in reponse_values:
    print (row)
    print(row["symbol"])

{'eps': '$0.27', 'surprise': '-3.57', 'time': 'time-not-supplied', 'symbol': 'BF.B', 'name': 'Brown Forman Corporation', 'marketCap': '$32,700,006,773', 'fiscalQuarterEnding': 'Apr/2020', 'epsForecast': '$0.28', 'noOfEsts': '6'}
BF.B
{'eps': '$0.27', 'surprise': '999', 'time': 'time-not-supplied', 'symbol': 'BF.A', 'name': 'Brown Forman Corporation', 'marketCap': '$29,319,555,714', 'fiscalQuarterEnding': 'Apr/2020', 'epsForecast': '', 'noOfEsts': '0'}
BF.A
{'eps': '($0.12)', 'surprise': '-9.09', 'time': 'time-not-supplied', 'symbol': 'CHWY', 'name': 'Chewy, Inc.', 'marketCap': '$19,928,902,131', 'fiscalQuarterEnding': 'Apr/2020', 'epsForecast': '($0.11)', 'noOfEsts': '6'}
CHWY
{'eps': '($0.53)', 'surprise': '-51.43', 'time': 'time-not-supplied', 'symbol': 'TIF', 'name': 'Tiffany & Co.', 'marketCap': '$14,687,613,128', 'fiscalQuarterEnding': 'Apr/2020', 'epsForecast': '($0.35)', 'noOfEsts': '3'}
TIF
{'eps': '($0.91)', 'surprise': '-160', 'time': 'time-not-supplied', 'symbol': 'FIVE', 'n

In [12]:
from mapper.predict_logic_mapper import MapperPredictLogic, simplejson

In [27]:
db = PgsqlExecutor(research_db)

In [32]:
mapper_logic = MapperPredictLogic(test_connect)

In [33]:
data = mapper_logic.get_dcf_data("bkh")
print(data, list(data))

[-1.462, -470.2, 0.0, -100.0, -100.0] [-1.462, -470.2, 0.0, -100.0, -100.0]


In [34]:
dto = mapper_logic.get_active_session('dcf')

In [35]:
dto = mapper_logic.get_dto(name='dcf')
dto = mapper_logic.select(dto, where_column='name')

In [36]:
dto.features

['last_growth_rate',
 'last_market_dcf',
 'last_dcf',
 'dcf_to_price',
 'market_dcf_to_price']

In [37]:
mapper_logic.predict('dcf', data)

-31.950874

In [38]:
for row in reponse_values:
    symbol = row['symbol']
    data = mapper_logic.get_dcf_data(symbol)
    if data:
        print (symbol, mapper_logic.predict('dcf', data))

TIF -21.95116
FIVE -12.868908
HDS 3.175607
VRNT 0.8183343
M -8.939942
AGX -3.0153992
MOV 13.464974
CONN -22.544954
LMNR -28.209805
ITI -27.706593
UEC -44.25919
LAKE -4.088923
MVC 3.5295596
VNCE -24.174091
PPIH -25.922293
ANIX -37.232117
CPST -33.46676


In [39]:
def get_prediction_by_earning_date(year, month, date):
    url = "https://api.nasdaq.com/api/calendar/earnings?date={}-{}-{}".format(
        year, month, date
    )
    response = requests.get(url)
    reponse_values = response.json()['data']['rows']
    for row in reponse_values:
        symbol = row['symbol']
        data = mapper_logic.get_dcf_data(symbol)
        if data:
            print (symbol, mapper_logic.predict('dcf', data))
    

In [40]:
get_prediction_by_earning_date("2020", "06", "11")

ADBE -52.46427
LULU -13.114918
PVH -65.43542
FLR 0.8766765
PLAY -24.455587
PLCE -5.8727036
XBIT -13.233391
TNP -16.336273
MARK -48.513958
RTIX -18.965872
INWK -28.602146
FUV -66.99868
SEAC -36.009686
CAAS -9.567344
RFIL -13.928219
ALOT -25.922293
AINC -3.4609754
UFAB 5.7043314


NameError: name 'db' is not defined